## 参考記事「Kaggleに登録したら次にやること ～ これだけやれば十分闘える！Titanicの先へ行く入門 10 Kernel ～」
Link : https://qiita.com/upura/items/3c10ff6fed4e7c3d70f0

## 1. まずはsubmit! 順位表に載ってみよう
kaggleではいつくかの方法で自分が作成した機械学習モデルの予測結果を提出(submit)できる(Notebook経由のみのコンペもあり).
* Notebook経由
* csvファイルを直接アップロード
* Kaggle APIを利用

## 2 全体像を把握！ submitまでの処理の流れを見てみよう
具体的な処理の流れ
1. パッケージの読み込み
2. データの読み込み
3. 特徴量エンジニアリング(Feature Engineering)
4. 機械学習アルゴリズムの学習・予測
5. 提出

### パッケージの読み込み
以降の処理で利用するパッケージの読み込みを行う.

In [1]:
import numpy as np # road matrix calculation library 
import pandas as pd # road data analysis library 

### データの読み込み
kaggleから提供されているデータを読み込む.
データの詳細はkaggleのコンペティションのページの「Data」タブや,右上のDataの項目から見ることができる.  
train.csvは機械学習の訓練用のデータです.test.csvは予測を実施するデータです.gender_submission.csvは提出のサンプルです.このファイルを見ると,提出ファイルの形式を確認できます.  
head関数を用いて実際にデータが読み込めていることを確認します.``train.head()``の実行結果を見ると,Pclass(搭乗しているクラス)やName(名前)など乗客一人一人の情報が表になっていることがわかります.
このコンペで予想するのはtestデータのSurvived(0:死亡,1:生存)という項目です.確認のために,``test.head()``の結果を見ると``Survived``の項目がないことがわかります.また``gender_submission.head()``
の実行結果を見ると``PassengerId``と``Survived``が記述されています.  
  
データをもう少し詳しくみると,``Name``や``Sex``などは文字列が格納されているため,そのままでは機械学習アルゴリズムの入力にすることができません.これらは機械学習アルゴリズムが扱える
形に変換する必要があります.また,``Nan``というのはデータの欠損です.欠損値は,一部の機械学習アルゴリズムではそのまま扱うこともできますが,代表値で穴埋めする場合もあります.

In [2]:
PATH = '../input/titanic/' # set PATH string
train = pd.read_csv(PATH+'train.csv') # read train data(csv)
test = pd.read_csv(PATH+'test.csv') # read train(csv)
gender_submission = pd.read_csv(PATH+'gender_submission.csv') # read submit sample

In [3]:
# 実際に読み込めているか確認
train.head() # trainの先頭5行を表示

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


### 特徴量エンジニアリング
次のような処理を特徴量エンジニアリングという.
* 読み込んだデータを機械学習アルゴリズムが扱える形に変換する.
* 既存のデータから,機械学習アルゴリズムが予測する上で有用な新しい特徴量を生成する.
前者については,例えば``Sex``の``male``,``female``をそれぞれ0と1に変換します.また欠損値を埋める処理も行います.後者については次のNotebookで詳しく掘り下げていきます.  
  
大元のデータから特徴量エンジニアリングを経て,X_train,y_train,X_testというデータを作ります.大雑把な表現をすると,X_train,y_trainの対応関係を学習し,
X_testに対応する(未知の)y_testの値を当てるという仕組みです.このような仕組みを「教師あり学習」と呼びます.(cf.教師なし学習)

In [6]:
dataset = pd.concat((train,test),ignore_index=True) # concat train and test
# print(pd.isnull(dataset).sum()) # the number of missing values in each feature
dataset.replace(['male','female'], [0, 1], inplace=True) # convert Sex into numbers
dataset['Embarked'] = dataset['Embarked'].fillna('S') # fill missing value int Embarked(mode)
dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].mean())
dataset.replace(['S','C','Q'], [0, 1,2], inplace=True) # convert Embarked into numbers
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [7]:
# remove unused features
drop_features = ['PassengerId','Name','Age','Ticket','Cabin']
dataset.drop(labels = drop_features, axis = 1, inplace = True)
# make X_train
X_train = dataset[:len(train)]
# make y_train
y_train = X_train['Survived']
X_train = X_train.drop(labels = 'Survived',axis = 1)
# make test
X_test = dataset[len(train):]
X_test = X_test.drop('Survived',axis=1)

### 機械学習アルゴリズムの学習・予測
用意した特徴量と予測の対象のペアから,機械学習アルゴリズムを用いて予測器を学習させましょう.
まず,予測器の読み込みを行います.次に読み込んだ予測器をインスタンス化します.そして``fit``関数を用いて学習を行います.最後に予測値が未知の特徴量(X_test)を与えて予測させます(``predict``関数).
y_predの中身は0と1の予測値が格納されています.

In [8]:
from sklearn.linear_model import LogisticRegression 

clf = LogisticRegression(penalty='l2', solver='sag', random_state=0) # instantiate LogisticRegression
clf.fit(X_train, y_train) # learning 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(random_state=0, solver='sag')

In [9]:
y_pred = clf.predict(X_test) # predict  

### 提出(submit)
最後にNotebook経由で提出するために予測値を提出ファイル形式に整えます.
下のプログラムまでエラーなく実行できたら右上の``Save Version`` -> ``Save & Run All (Commit)``を選択 -> ``Save``をクリック

In [10]:
submit = pd.DataFrame(test['PassengerId'])
submit['Survived'] = list(map(int, y_pred))
submit.to_csv('submission.csv', index=False)